# Generate Random Models

In testing, we often need to generate random models. `schematics` offer this capability via a feature called [Model Mocking]



[Model Mocking]: https://schematics.readthedocs.io/en/latest/usage/models.html#model-mocking

In [1]:
import schematics

class EnvironmentMetadata(schematics.Model):
    name = schematics.types.StringType(required=True, max_length=30)
    display_name = schematics.types.StringType(serialized_name="displayName", max_length=255, serialize_when_none=False)
    description = schematics.types.StringType(max_length=255, serialize_when_none=False)

In [2]:
environment_metadata = EnvironmentMetadata.get_mock_object()

In [3]:
environment_metadata.name

'7fUFQNnToFt7qvnf'

In [4]:
environment_metadata.display_name

In [5]:
environment_metadata.description

In [6]:
environment_metadata.to_primitive()

{'name': '7fUFQNnToFt7qvnf'}

## Generate Nested Models

In [7]:
import json
import random

import schematics

class UnixUser(schematics.Model):
    alias = schematics.types.StringType(required=True)
    is_admin = schematics.types.BooleanType(default=False)
    
class Phone(schematics.Model):
    number = schematics.types.StringType(regex=r"\d{3}-\d{3}-\d{4}", required=True)
    kind = schematics.types.StringType(required=True,choices=["mobile", "work", "home", "other"])
    
class Contact(schematics.Model):
    name = schematics.types.StringType(required=True)
    
    # Nested: A single object
    unix_user = schematics.types.ModelType(UnixUser, serialized_name="unixUser", required=True)
    
    # Nested: Many objects
    phones = schematics.types.ListType(schematics.types.ModelType(Phone), default=[])

In [8]:
for _ in range(3):
    contact = Contact.get_mock_object()
    print("-" * 72)
    print(json.dumps(contact.to_primitive(), indent=4))

------------------------------------------------------------------------
{
    "name": "N2PBJEsTx83ICQ",
    "unixUser": {
        "alias": "6nkKX",
        "is_admin": false
    },
    "phones": [
        {
            "number": "vO34FoabI",
            "kind": "mobile"
        },
        {
            "number": "6wBjol0rIx9MzqX2",
            "kind": "other"
        },
        {
            "number": "HjAjN",
            "kind": "other"
        },
        {
            "number": "NgFDg",
            "kind": "home"
        },
        {
            "number": "LDmATu",
            "kind": "mobile"
        },
        {
            "number": "tt",
            "kind": "work"
        },
        {
            "number": "88K0",
            "kind": "home"
        },
        {
            "number": "PGOETPHvfV",
            "kind": "other"
        },
        {
            "number": "EH7zGh58w6Yat7R",
            "kind": "work"
        },
        {
            "number": "Fea",
            "kind"

## Overriding the Random Generator

If we need to override some fields, we can create a dictionary and pass that into `get_mock_object()`:

In [9]:
contact = Contact.get_mock_object(overrides={"name": "Johan"})
print(json.dumps(contact.to_primitive(), indent=4))

{
    "name": "Johan",
    "unixUser": {
        "alias": "uRSrDai",
        "is_admin": true
    },
    "phones": [
        {
            "number": "Nr",
            "kind": "other"
        },
        {
            "number": "FdtU6CGL4G",
            "kind": "other"
        },
        {
            "number": "",
            "kind": "work"
        },
        {
            "number": "WoqgiyUm",
            "kind": "work"
        },
        {
            "number": "47JKpi26RS",
            "kind": "work"
        },
        {
            "number": "GRIxW8",
            "kind": "work"
        },
        {
            "number": "z3DbOx5XL0c",
            "kind": "work"
        }
    ]
}


## Caveats

Note that in the example above, the random models might not be valid. For example, the phone number field does not satisfy the regular expression:

In [10]:
phone = Phone.get_mock_object()
print(phone.to_primitive())
try:
    phone.validate()
except schematics.exceptions.DataError as error:
    print(f"Error: {error}")

{'number': 'g19', 'kind': 'work'}
Error: {"number": ["String value did not match validation regex."]}


For this reason, if we want to generate our own valid models, we need to do it ourselves and do not rely on `get_mock_object`.

In [11]:
def generate_random_phone_number():
    number = "-".join(
        str("".join(random.sample("0123456789", n)))
        for n in [3, 3, 4]
    )
    return number

class Phone(schematics.Model):
    number = schematics.types.StringType(regex=r"\d{3}-\d{3}-\d{4}", required=True)
    kind = schematics.types.StringType(required=True,choices=["mobile", "work", "home", "other"])
    
    @classmethod
    def get_mock_object(cls, context=None, overrides=None):
        obj = super().get_mock_object(context=context, overrides=overrides or {})
        try:
            obj.validate()
        except schematics.exceptions.DataError:
            obj.number = generate_random_phone_number()
        return obj

In [12]:
for _ in range(5):
    phone = Phone.get_mock_object()
    print(phone.to_primitive())
    phone.validate()

{'number': '197-328-0849', 'kind': 'home'}
{'number': '390-094-2869', 'kind': 'home'}
{'number': '971-729-0687', 'kind': 'mobile'}
{'number': '043-487-9456', 'kind': 'home'}
{'number': '674-381-8912', 'kind': 'work'}


In testing, sometimes we need invalid random models, which means the above method will not satisfy. In which case it helps to keep the inherited `get_mock_object()` method untouch and create a separate generator.

In [13]:
class Phone(schematics.Model):
    number = schematics.types.StringType(regex=r"\d{3}-\d{3}-\d{4}", required=True)
    kind = schematics.types.StringType(required=True,choices=["mobile", "work", "home", "other"])
    
    @classmethod
    def get_valid_mock_object(cls, context=None, overrides=None):
        obj = cls.get_mock_object(context=context, overrides=overrides or {})
        obj.number = generate_random_phone_number()
        return obj

In [14]:
for _ in range(5):
    phone = Phone.get_valid_mock_object()
    print(phone.to_primitive())
    phone.validate()

{'number': '910-129-5478', 'kind': 'mobile'}
{'number': '934-537-6172', 'kind': 'mobile'}
{'number': '267-794-6825', 'kind': 'other'}
{'number': '931-029-8029', 'kind': 'home'}
{'number': '483-693-2859', 'kind': 'mobile'}


## Summary

* Generate a random model with `Class.get_mock_object()`
* Override some field with the `get_mock_object(orverrides={some-dict})`
* The generated model might not be valid
* To ensure a valid model, write our own version of `get_mock_object()`